A text summarization example for GPT-2 (not sure this implementation is completely correct).  
The data is XSum from https://github.com/EdinburghNLP/XSum.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install nlp
!pip install rouge

In [ ]:
%tensorflow_version 2.x

import os
import warnings
import time
import re
import nlp
import pprint
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from rouge import Rouge

os.chdir('./drive/My Drive/Python/Research/bert')
warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import mymodels as mm

In [ ]:
MAXLEN = 256
BATCH = 16
EPOCH = 2
LRATE = 1e-4
CONFIG = './models/gpt_base_en/hparams.json'
CKPT = './models/gpt_base_en/model.ckpt'
VOCAB = './models/gpt_base_en/encoder.json'
SAVE = './tasks/models/XSum/model'

Load and process the data. Use short documents only.

In [ ]:
def file_processing(file, droplen):
  file['length'] = file.apply(lambda x: len(x['document'].split(' '))+len(x['summary'].split(' ')), axis=1)
  return file[file['length'] < droplen*0.9].reset_index(drop=True)


def data_processing(data, tokenizer, maxlen, batch, training):
  text1, label1, mask1, mlen1 = [], [], [], maxlen+1
  pad1 = tokenizer.vocab[tokenizer.pad]

  for i1 in range(len(data)):
    sum1, doc1 = re.sub('\n', '', data['summary'][i1]), data['document'][i1]
    t1, _, _, l1 = tokenizer.encoding(sum1, None, mlen1, bos=False, pad=False, length=True)
    t2, _, _, l2 = tokenizer.encoding(doc1, None, mlen1-l1, bos=False, pad=False, length=True)
    t1 = t2+t1+[pad1]*(mlen1-l1-l2)
    m1 = [0.]*(l2-1)+[1.]*l1+[0.]*(mlen1-l1-l2)
    text1, label1, mask1 = text1+[t1[:-1]], label1+[t1[1:]], mask1+[m1]

  text1, label1, mask1 = np.array(text1), np.array(label1), np.array(mask1)
  data1 = tf.data.Dataset.from_tensor_slices((text1, label1, mask1))
  return data1.shuffle(len(text1)).batch(batch) if training else data1.batch(batch)


data_file = nlp.load_dataset('xsum')
training_file = file_processing(pd.DataFrame(data_file['train']), MAXLEN)
dev_file = file_processing(pd.DataFrame(data_file['validation']), MAXLEN)
test_file = file_processing(pd.DataFrame(data_file['test']), MAXLEN)

gpt_tokenizer = mm.Tokenizer(False, False, False)
gpt_tokenizer.loading(VOCAB)
training_set = data_processing(training_file, gpt_tokenizer, MAXLEN, BATCH, False)
dev_set = data_processing(dev_file, gpt_tokenizer, MAXLEN, BATCH, False)
training_file.head()

Using custom data configuration default


,document,summary,length
0,The problem is affecting people using the olde...,Sony has told owners of older models of its Pl...,201
1,"Only Men Aloud, winners of the BBC's Last Choi...",The winners of a TV talent show have beaten th...,174
2,The National Oceanic and Atmospheric Administr...,As many as 14 hurricanes could hit the Atlanti...,116
3,The 50-year-old musician was treated at a spec...,U2's lead singer Bono has had emergency spinal...,218
4,"Jordan Romero, from California, telephoned his...",A 13-year-old American boy has become the youn...,210


Load GPT-2 and prepare for training.

In [ ]:
class GPT2(mm.GPT):
  def __init__(self, config):
    super(GPT2, self).__init__(config)

  @tf.function(experimental_relax_shapes=True)
  def iterating(self, x, mask, pos, past, training, softmax):
    return super(GPT2, self).propagating(x, mask, pos, past, training, softmax)

  def propagating(self, x, mask=None, pos=None, past=None, training=False, softmax=False):
    return self.iterating(x, mask, pos, past, training, softmax) if past is not None else super(
      GPT2, self).propagating(x, training=training, softmax=softmax)


class LM(keras.Model):
  def __init__(self, tokenizer, gpt, **kwargs):
    super(LM, self).__init__(**kwargs)
    self.tokenizer, self.gpt = tokenizer, gpt

  def propagating(self, x, training, softmax):
    return self.gpt.propagating(x, training=training, softmax=softmax)
  
  def tokenizing(self, text, maxlen):
    text1, mask1, length1 = [], [], []

    for i1 in text:
      t1, s1, m1, l1 = self.tokenizer.encoding(i1, None, maxlen, bos=False, length=True)
      text1, mask1, length1 = text1+[t1], mask1+[m1], length1+[[l1]]

    text1, mask1, length1 = np.array(text1), np.array(mask1), np.array(length1)
    return text1[:, :int(max(length1))], mask1[:, :int(max(length1))], length1-1

  def generating(self, text, beam=5, k=1, p=0.9, temp=1.0, penalty=1.0, orilen=256, genlen=64, minlen=2):
    token1, mask1, length1 = self.tokenizing(text, orilen)
    gen1, dec1 = self.gpt.generating(token1, mask1, length1, beam, k, p, temp, penalty, genlen, False), []

    for i1 in gen1:
      curlen1, cur1, out1 = 0, None, None

      for j1 in i1:
        leng1 = len(j1[:(j1+[self.gpt.eos]).index(self.gpt.eos)])

        if leng1 >= minlen:
          out1 = self.tokenizer.decoding(j1)
          break

        if leng1 > curlen1:
          curlen1, cur1 = leng1, self.tokenizer.decoding(j1)

      dec1 = dec1+[cur1] if out1 is None else dec1+[out1]

    return dec1, gen1


@tf.function
def step_training(x, y, mask):
  with tf.GradientTape() as tape1:
    pred1 = lm_model.propagating(x, True, True)[0]
    mask1 = tf.cast(mask, tf.float32)
    loss1 = tf.reduce_sum(loss_function(y, pred1)*mask1)/tf.reduce_sum(mask1)

  grad1 = tape1.gradient(loss1, lm_model.trainable_variables)
  grad1, _ = tf.clip_by_global_norm(grad1, 1.0)
  adam_optimizer.apply_gradients(zip(grad1, lm_model.trainable_variables))
  training_loss(loss1)
  training_acc(y, pred1, sample_weight=mask1)


@tf.function
def step_evaluating(x, y, mask):
  pred1 = lm_model.propagating(x, False, True)[0]
  mask1 = tf.cast(mask, tf.float32)
  dev_acc(y, pred1, sample_weight=mask1)


gpt_model = GPT2(CONFIG)
gpt_model.loading(CKPT)
lm_model = LM(gpt_tokenizer, gpt_model)
loss_function = keras.losses.SparseCategoricalCrossentropy(reduction=keras.losses.Reduction.NONE)
adam_optimizer = mm.AdamWV2(EPOCH*(len(training_file)/BATCH+1), LRATE)

training_loss = tf.keras.metrics.Mean(name='training_loss')
training_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='training_accuracy')
dev_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='dev_accuracy')

Train the model. Here only consider loss and accuracy.

In [ ]:
log_info = 'Training loss is {:.4f}, and accuracy is {:.4f}.'
end_info = 'Dev accuracy is {:.4f}, and epoch cost is {:.4f}.'

for epoch_count in range(EPOCH):
  print('Epoch {} running.'.format(epoch_count+1))
  step_count, time_point = 0, time.time()

  for training_x, training_y, training_mask in training_set:
    step_training(training_x, training_y, training_mask)
    step_count = step_count+1

    if step_count % 1000 == 0:
      print(log_info.format(float(training_loss.result()), float(training_acc.result())))

  for dev_x, dev_y, dev_mask in dev_set:
    step_evaluating(dev_x, dev_y, dev_mask)

  print(end_info.format(float(dev_acc.result()), time.time()-time_point))
  print('**********')
  training_acc.reset_states()
  dev_acc.reset_states()
  lm_model.save_weights(SAVE)

Epoch 1 running.
Training loss is 2.5463, and accuracy is 0.4927.
Training loss is 2.4302, and accuracy is 0.5081.
Training loss is 2.3558, and accuracy is 0.5186.
Training loss is 2.3042, and accuracy is 0.5268.
Dev accuracy is 0.5553, and epoch cost is 3255.1307.
**********
Epoch 2 running.
Training loss is 2.2051, and accuracy is 0.5870.
Training loss is 2.1074, and accuracy is 0.6104.
Training loss is 2.0399, and accuracy is 0.6176.
Training loss is 1.9933, and accuracy is 0.6202.
Dev accuracy is 0.5656, and epoch cost is 3233.4219.
**********


Evaluate the model on test set using ROUGE score.

In [ ]:
lm_model.load_weights(SAVE)
test_hyp, test_ref, test_doc = [], test_file['summary'].tolist(), test_file['document'].tolist()

for range_id in range(int(np.ceil(len(test_file)/BATCH))):
  inp_doc = test_doc[range_id*BATCH:(range_id+1)*BATCH]
  test_hyp += lm_model.generating(inp_doc, orilen=MAXLEN, genlen=40, minlen=4)[0]

rouge_score = Rouge()
rouge_score.get_scores(test_hyp, test_ref, avg=True)

{'rouge-1': {'f': 0.3117434702731899,
  'p': 0.3769333529610374,
  'r': 0.2772046486630661},
 'rouge-2': {'f': 0.12555274489288518,
  'p': 0.1520535098089593,
  'r': 0.11167880148199814},
 'rouge-l': {'f': 0.2723238256710355,
  'p': 0.3291359575144691,
  'r': 0.24143634070716324}}

Check some summarization results from GPT-2.

In [ ]:
sample_doc = test_file['document'][1928]
pprint.pprint(sample_doc)
lm_model.generating([sample_doc], orilen=MAXLEN, genlen=40, minlen=4)[0][0]

('The victims were farming in an area that was declared unsafe because of its '
 'close proximity to Mount Sinabung.The volcano was still spewing ash on '
 'Sunday, hampering rescue operations.More than a dozen people were killed '
 'when it erupted in 2014. It also erupted in 2010, after having been dormant '
 'for 400 years.Rescue teams are still scouring the area, looking for more '
 'victims who may have been killed or badly burned by the hot gas and ash '
 'clouds released in the eruption.Rescue teams were searching homes and farms '
 'in the village of Gamber, which was also evacuated in 2014.What causes '
 "volcanoes?The 2,460-metre (8,070 foot) tall volcano is among the country's "
 'most active.Indonesia, located on the Pacific Ring of Fire, has more than '
 '120 active volcanoes.\n')


' Rescue teams in Indonesia are searching for survivors after a volcano erupted in the capital Jakarta.'